In [82]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cadd_io as cdio
import cadd_main as cdmain
import mdutilities_io as mduio
import mymath as Mmath
import copy
import my_plot as myplot
import cadddatadump as cddump
import cadd_mesh2 as cdmesh
import cadd
import itertools
import newpotential as newpot
import tabularpotential as tabpot
import crystallography_oo as croo
import lineplot_oo3 as lpoo
%matplotlib qt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Preliminaries

## Directories

In [83]:
maindir = '../Tests/CADD_K_Test/'
uidir = maindir + 'User Inputs/'
fidir = maindir + 'Fortran Inputs/'
dumpdir = maindir + 'Dump Files/'
simname = 'cadd_k_test'
simtype = 'cadd'

In [84]:
size = 'large'
if size == 'small':
    ferings = 5
elif size == 'medium':
    ferings = 14
elif size == 'large':
    ferings = 24
simname += '_' + size

## Unit System

In [85]:
# LJ
r0 = 1.0
Alr0 = 125e-12
Almass = (27)/(6.02e23)
# Alcohenergy = 5.4e-19 # cohesive energy/atom
Almu = 28e12
Altime = np.sqrt(Almass/(Almu*Alr0))
myr0 = r0
mymass = 1.0
mymu = 13.4
# mycohenergy = 6.0
mytime = np.sqrt(mymass/(mymu*myr0))
lengthfac = myr0/Alr0
timefac = mytime/Altime
massfac = mymass/Almass
stressfac = massfac/(lengthfac*timefac**2)

In [86]:
1e-4*stressfac*timefac

0.00011549009898239314

# Create Inputs

## Nodes/Elements

### Create Mesh

In [87]:
padx, pady = 5, 5
lx, ly = 42, 45 # gives roughly 2500 atoms (including pad, interface)
xy = cadd.simple_hex_lattice(lx+padx,ly+pady,r0=r0) # make box slightly larger to accommodate pad
meshnodes = cdmesh.Nodes(xy)
mesh = cdmesh.MeshTri(nodes=meshnodes)
meshnodes = mesh.nodes
interfacebox = meshnodes.gen_interface_and_pad(np.array([-lx/2,lx/2,-ly/2,ly/2]))
boxold = copy.copy(interfacebox)
for i in range(ferings):
    nnodesvec = [line.get_nnodes() for line in boxold.lines]
    if i < 10:
        if i%3 == 0:
            ring = cdmesh.RingCollapseMixed(boxold)
        else:
            ring = cdmesh.RingExtend(boxold)
    else:
        ring = cdmesh.RingCollapseMixed(boxold)
    mesh.add_ring(ring)
    boxold = ring.boxnew

In [88]:
# k-field will be imposed on outer boundary
outernodes = boxold.all_nodes
meshnodes.set_node_bc(outernodes,3) # outer nodes are completely fixed

### Fix Mesh to Account for Crack Plane

In [89]:
# crack line and nodes
idx = np.argmin(np.abs(meshnodes.xy[:,1]))
ycrack = meshnodes.xy[idx,1]
crackline = cdmesh.Line([-np.inf,ycrack],[0.0,ycrack])
nodenums = meshnodes.search_for_nodes_along_line(crackline)

In [90]:
# construct new nodes
# the bit with ydisp is a bit kludgy, and might depend on the specific mesh...
mapping = {}
ydisp = r0*np.sqrt(3)/2 # atomic spacing in y-direction
ydispvec = np.array([0,ydisp])
ydispvecatom = np.array([r0/2,ydisp])
defaultfetype = cdmesh.Nodes.defaultfetype
for nodenum in nodenums:
    posn = meshnodes.xy[nodenum,:]
    nodetype = meshnodes.types[nodenum,:]
    if nodetype[1] in [0,2]: # FE node or interface atom
        if nodetype[1] == 2: # interface atom must be moved
            posnnew = posn + ydispvecatom
            nodenew = meshnodes.closest_node(posnnew)
        else:
            posnnew = posn + ydispvec
            nodenew = meshnodes.add_node(posnnew,defaultfetype)
        meshnodes.types[nodenew,:] = nodetype # retain type and bc of old node
        mapping[nodenum] = nodenew  

In [91]:
# this modifies the element while it's being looped over, which is dangerous
for i, element in enumerate(mesh.elements):
    common = np.intersect1d(element,nodenums,assume_unique=True)
    if common.size: # nodenum belongs to element
        pts = meshnodes.get_points(element)
        yelementcenter = np.mean(pts,axis=0)[1]
        if Mmath.same_sign(yelementcenter-ycrack,ydisp): # only create a new element if the element center is on one side of ycrack
            for nodeold in common:
                nodenew = mapping[nodeold]
                np.place(element,element==nodeold,nodenew) # replace old num with new num

In [92]:
# fudge fe node positions slightly to ensure pad atoms are within an element
yfudge = ydisp/20
ydispvec = np.array([0,np.copysign(yfudge,ydisp)])
for nodeold, nodenew in mapping.items():
    if meshnodes.types[nodeold,1] == 0: # not an interface node (which should remain fixed)
        meshnodes.xy[nodeold,:] += ydispvec
        meshnodes.xy[nodenew,:] -= ydispvec

### Plot

In [93]:
mesh.write_dump_all('test.6.dump')
sim = cdmain.Simulation('cadd_nodisl','test',readinput=False)
# fig = sim.plot_dump_from_file('test.6.dump',axisbounds=interfacebox.bounds)

### Nodes

In [94]:
n = meshnodes.xy.shape[0]
z = np.zeros((n,))
posn = np.column_stack((meshnodes.xy,z)) # z-coordinate = 0
nodes = cdmain.Nodes(posn=posn,types=meshnodes.types)

### FE Elements

In [95]:
feelements = cdmain.FEElement(elname='CPE3',mnum=1,
                              connect=mesh.elements_dump) # elements_dump fixes off by 1 indexing issue

## Potential

In [96]:
inter1new = newpot.gen_std_potential(ductilityindex=5,dimensions=2,r0=1)

In [97]:
rminsmall, rmin, rmax = 0.01, 0.1, 2.0
npoints = 3000
rvec = np.linspace(rmin,rmax,npoints)
rvec = np.insert(rvec,0,rminsmall)
pottable = inter1new.get_energy_force_table(rvec)

In [98]:
inter1new.write_file_lammps('inter1newnorm',np.linspace(0.1,3.0,3000),extend=True)

In [99]:
# Energy plot
# lpoo.my_quick_plot([pottable[:,[0,1]]],axisbounds=[0.5,2,-1.1,3])

In [100]:
rnnn = np.sqrt(3)*r0 # 2nd nearest neighbor
potential = cdmain.Potential(pname='inter1new',forcecutoff=rnnn,pottable=pottable)

## Material

### Elastic constants

In [101]:
elasticdict = inter1new.elasticdict('hex')
mymat = croo.Material('hex',elasticdict)
elconst = mymat.voigt_plane_strain_stiffness

### Dislocation stuff

In [102]:
lannih = 6*r0
aluminumdislvmax = 1000
mydislvmax = aluminumdislvmax*(lengthfac/timefac)
aluminumdisldrag = 1e-4
mydisldrag = aluminumdisldrag*(stressfac*timefac)

In [103]:
material = cdmain.Material(burgers=r0,disldrag=0.0,dislvmax=0.0,elconst=elconst,lannih=6*r0,lattice='hex',mass=1.0,
                           mname='material1',rho=rho)

## Misc

In [104]:
misc = cdmain.Misc(increments=100,timestep=0.02,iscrackproblem=True,potstyle='table')

## Interactions

In [105]:
interactions = cdmain.Interactions(table=np.array([[1,1,1]]))

## Neighbors

In [106]:
neighbors = cdmain.Neighbors(checkdisp=True,every=0,delay=5,images=0,Lz=1.0,skin=0.3)

## Damping

In [107]:
gamma = 0.1 # may need to fiddle with this
damping = cdmain.Damping(flag=True,gname='all',gamma=gamma)

## DD Stuff

In [108]:
# no dislocations, sources, or obstacles
escapeddisl = cdmain.EscapedDislocations()
ghostdisl = cdmain.GhostDislocations()
disl = cdmain.Dislocations()
sources = cdmain.Sources()
obstacles = cdmain.Obstacles()

### Slip Systems

In [109]:
# just a few slip planes accessible from atomistic region (i.e., close to the origin)
# fudges are to ensure full coverage
nslipsys = 3
xmin, xmax, ymin, ymax = interfacebox.bounds
fudge, fudge2 = 1.1, 1.5
planespacing = r0*np.sqrt(3)/2
maxdist = fudge*np.sqrt((xmax-xmin)**2 + (ymax-ymin)**2)
nplanes = fudge2*int(round(maxdist/planespacing))

In [110]:
# recall that origin is at very edge of slip system --- slip planes should be built up in only one direction
theta = np.empty((nslipsys,))
origin = np.empty((nslipsys,2))
nslipplanes = nplanes*np.ones((nslipsys,))
spacing = planespacing*np.ones((nslipsys,))

#### System 1 (theta = 0)

In [111]:
theta[0] = 0.0
origin[0,:] = np.array([0,fudge*ymin])

#### System 2 (theta = 60)

In [112]:
theta[1] = np.pi/3
origin[1,:] = np.array([fudge*xmax,fudge*ymin])

#### System 3 (theta = -60)

In [113]:
theta[2] = -np.pi/3
origin[2,:] = np.array([fudge*xmin,fudge*ymin])

In [114]:
slipsys = cdmain.SlipSystem(theta=theta,origin=origin,nslipplanes=nslipplanes,space=spacing)

AttributeError: 'SlipSystem' object has no attribute '_norigincheck'

## Detection

### Interface Edges

In [ ]:
def interface_edges(meshnodes,ydispvec,interfaceboxactual):
    nnodes = meshnodes.nnodes
    auxlist = np.arange(0,nnodes)
    interfacenodesidx = meshnodes.types[:,1] == 2
    xynew = meshnodes.xy[interfacenodesidx,:]
    auxlistnew = auxlist[interfacenodesidx]
    def closest_node(pt):
        dist = np.linalg.norm(xynew - pt, axis=1)
        idx = np.argmin(dist)
        return auxlistnew[idx]
    ptsall = [[line.startpoint,line.endpoint] for line in interfacebox.lines]
    interfaceedges = [[closest_node(pt) for pt in pts] for pts in ptsall]
    return np.array(interfaceedges)

In [ ]:
interfaceedges = interface_edges(meshnodes,ydispvec,interfacebox)

### Detection Annulus

In [ ]:
bandtype = 'rect_annulus'
ptsall = np.array([line.startpoint for line in interfacebox.lines])
boxcenter = np.mean(ptsall,axis=0)
_, xmax, _, ymax = interfacebox.bounds
halfLxouter = xmax - boxcenter[0]
halfLyouter = ymax - boxcenter[1]

In [ ]:
xthickness, ythickness = 5, 5
halfLxinner = halfLxouter - xthickness
halfLyinner = halfLyouter - ythickness
passdistance = np.sqrt(xthickness**2 + ythickness**2)

In [ ]:
params = np.array([boxcenter[0],boxcenter[1],halfLxinner,halfLxouter,halfLyinner,halfLyouter])

### Other

In [ ]:
# may have to fiddle with these
dampgamma = 0.1
detectiondamp = cdmain.Damping.temp_damping(gamma=dampgamma)
mdnincrements = 500
mdtimestep = 0.02
mnumfe = 1

### All

In [ ]:
detection = cdmain.Detection(bandtype=bandtype,damp=detectiondamp,interfaceedges=interfaceedges,
                             mdnincrements=mdnincrements,mdtimestep=mdtimestep,mnumfe=mnumfe,
                             params=params,passdistance=passdistance)

## Everything

In [ ]:
cadddata = cdmain.CADDData(simtype,nfematerials=1,nodes=nodes,materials=[material],misc=misc,groups=None,
                               potentials=[potential],interactions=interactions,neighbors=neighbors,damping=damping,
                               feelements=[feelements],escapeddisl=[escapeddisl],ghostdisl=[ghostdisl],disl=[disl],
                               sources=[sources],obstacles=[obstacles],slipsys=[slipsys],detection=detection)
caddsim = cdmain.Simulation(simtype,simname,uidir,fidir,dumpdir,data=cadddata)
caddsim.data.check_data()

## Write

In [ ]:
caddsim.write_fortran_all()

# Misc

# Dump Files

In [ ]:
fig = caddsim.plot_dump_from_increment(7,tight=False,fignum=1,axisbounds=[-5,5,-5,5]);

In [ ]:
fig = caddsim.plot_dump_from_increment(9,tight=False,fignum=2,axisbounds=[-30,30,-30,30]);

In [ ]:
brittlenew.rho